<a href="https://colab.research.google.com/github/Kusuma-1309/Neural-Networks-and-Deep-Learning/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [5]:
pip install ultralytics


In [6]:
# object_detection_yolov8.py
# Simple object detection using Ultralytics YOLO (v8)
# Usage: python object_detection_yolov8.py input.jpg output.jpg

import sys
import cv2
from ultralytics import YOLO  # pip install ultralytics
import numpy as np

def draw_boxes(image, boxes, confidences, class_ids, names):
    """Draw labelled bounding boxes on the image (in-place)."""
    for (x1, y1, x2, y2), conf, cls in zip(boxes, confidences, class_ids):
        x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
        label = f"{names[int(cls)]}: {conf:.2f}"
        # choose a color based on class id
        color = tuple(int(c) for c in np.random.RandomState(int(cls)).randint(0, 255, size=3))
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        # text background
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image, (x1, y1 - th - 6), (x1 + tw, y1), color, -1)
        cv2.putText(image, label, (x1, y1 - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

def main(in_path, out_path):
    # Load pre-trained YOLOv8n model. The first run will download weights (~few MB).
    model = YOLO('yolov8n.pt')  # 'n' = nano, fast. Change to 'yolov8s.pt' for slightly larger model.

    # Run inference (returns list of results for each image)
    results = model(in_path, imgsz=640, conf=0.25, device='cpu')  # use device='0' for GPU if available

    # Read original image with OpenCV (BGR)
    img = cv2.imread(in_path)
    if img is None:
        raise FileNotFoundError(f"Could not open {in_path}")

    detected = []
    # results[0] corresponds to our single image
    r = results[0]
    # r.boxes is a structure with .xyxy, .conf, .cls
    if hasattr(r, 'boxes') and len(r.boxes) > 0:
        boxes = r.boxes.xyxy.cpu().numpy()      # shape (N,4): x1,y1,x2,y2
        confs = r.boxes.conf.cpu().numpy()      # shape (N,)
        cls_ids = r.boxes.cls.cpu().numpy()     # shape (N,)
        names = model.names                     # dict: id->name

        # draw boxes on the image
        draw_boxes(img, boxes, confs, cls_ids, names)

        # print summary
        for cid, conf in zip(cls_ids, confs):
            detected.append((names[int(cid)], float(conf)))

    else:
        print("No objects detected.")

    # Save annotated image
    cv2.imwrite(out_path, img)
    print(f"Saved result to {out_path}")

    # Print detected objects summary
    if detected:
        print("Detected objects (class, confidence):")
        for name, conf in detected:
            print(f" - {name}: {conf:.2f}")
    else:
        print("No objects detected.")

# Direct call instead of sys.argv
main("room.jpg", "output.jpg")




image 1/1 /content/room.jpg: 448x640 5 chairs, 2 couchs, 1 potted plant, 1 tv, 198.1ms
Speed: 3.7ms preprocess, 198.1ms inference, 2.4ms postprocess per image at shape (1, 3, 448, 640)
Saved result to output.jpg
Detected objects (class, confidence):
 - potted plant: 0.87
 - couch: 0.86
 - couch: 0.81
 - tv: 0.69
 - chair: 0.65
 - chair: 0.60
 - chair: 0.44
 - chair: 0.37
 - chair: 0.28
